In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp cli

# CLI 
> Contains functions for calling Rough from the command line

In [ ]:
#| export
from fastcore.script import *
import numpy as np
import pandas as pd
from pathlib import Path

from rough.data import *
from rough.profile import *
from rough.section import *

import rough.profile as profile_mod
import rough.section as section_mod

In [ ]:
array = np.loadtxt('example.txt')
array.shape

(501, 501)

In [ ]:
#| export
def compute_parameters(array, #Input array to be calculate paramers on
                       parameter_list:list, #List of parameters to calculate as strings
                       valid_module  = None, #module to generate functions from, used to check user input, see rough.cli:rough
                       to_df:bool = False, #Return the parameters as a pandas dataframe, with columns set as the parameter names
                       **kwargs #Keyword arguments to modify behavior of parameter calls, usually to define sections = True
                      ):
    
    results = []
    
    #The following generates a {'func':func} dict from given list of parameters if the parameter is available in the module
    valid_dict = {k: v for k, v in vars(valid_module).items() if callable(v) and k in valid_module.__all__}
    for parameter in parameter_list:
        result = valid_dict[parameter](array, **kwargs)
        results.append(result)
        
    if to_df:
        results_array = np.array(results)
        if len(results_array.shape) == 1: results_array = np.expand_dims(results_array, axis=1) #Fix for when only 1 section is being calculated
        return pd.DataFrame(data = results_array.T, columns = parameter_list)
    else:
        return results

In [ ]:
profile_mod.__all__

['Ra', 'Rms', 'Rsk', 'Rku']

In [ ]:
test_array = np.loadtxt('example.txt')
test_results =  compute_parameters(test_array, 
                                   parameter_list = profile_mod.__all__, 
                                   valid_module = profile_mod,
                                  to_df=True)

In [ ]:
test_sections = np.load('example_sections.npy')
section_results = compute_parameters(test_sections,
                                     parameter_list = section_mod.__all__,
                                     valid_module = section_mod,
                                     sections= True,
                                     to_df=False)


In [ ]:
test_section_result = compute_parameters(test_array, 
                                          parameter_list = section_mod.__all__,
                                          valid_module = section_mod,
                                          sections = False,
                                          to_df = True)

In [ ]:
test_section_result

,Sa,Sms,Ssk,Sku
0,0.000477,0.000633,0.095001,1.705203


In [ ]:
np.array(section_results).T.shape

(100, 4)

In [ ]:
test_results

,Ra,Rms,Rsk,Rku
0,0.000421,0.000523,-0.341025,-0.675181
1,0.000415,0.000512,-0.341625,-0.712358
2,0.000404,0.000496,-0.337859,-0.751569
3,0.000392,0.000479,-0.322040,-0.764241
4,0.000376,0.000462,-0.284401,-0.742131
...,...,...,...,...
496,0.001152,0.001425,-0.361214,-0.456548
497,0.001173,0.001452,-0.383951,-0.449050
498,0.001196,0.001483,-0.393435,-0.454725
499,0.001224,0.001520,-0.378328,-0.472839


In [ ]:
#| export
@call_parse
def rough(
    fname:str   = None,   #File name, path or directory with data files to be read
    ext:str     = '.txt', #Extension for the  files .txt or .csv
    
    result:str     = None,     #Directory to write results to, if None, writes to 'results'
    result_how:str = 'concat', #How to save the results, 'concat' concatenates all respective types of results 
                               #(i.e. profile,section,rotational,subsection) into one dataframe file.
                               #'split' produces respective result files for each input file. Use split for large amounts of data.
    
    level:bool  = True, #Perform plane levelling 
    form:bool   = True, #Remove form by polynomial subtraction
    deg:int     = 3,    #Degree of polynomial to remove
    smooth:bool = True, #Smooth the array by applying a gaussian
    sigma:int   = 1,    #Sigma for gaussian to be applied
    
    gen_rot:bool= True, #Generate rotational profiles and apply parameter calculation to them
    
    gen_section:bool= True, #Generate sub-sections of the surface
    sec_how:str = 'square', #Type of section to generate, currently only supports 'square'
    sec_num:int = 100,      #Number of sections to generate
    
    profile:bool = True, #Calculate profile parameters
    section:bool = True, #Calculate section parameters
    
    params1D:list = profile_mod.__all__, # list of 1D parameters to calculate,
    params2D:list = section_mod.__all__, #list of 2D parameters to calculate, calculates for both the sections and the whole
):
    '''
    This is intended to be the primary method used from the command line.
    '''
    
    delims = {'.txt': None,
              '.csv': ','}
    
    path       = Path().cwd() if fname == None else Path(fname)
    
    #Figuring out where the file/s are and results go
    if   path.is_dir():
        result_dir = path / 'results' if result == None else Path(result)
    elif path.is_file():
        result_dir = path.parent / 'results' if result == None else Path(result) 

    if not path.exists(): 
        raise FileNotFoundError('Could not find file/directory check fname')
    if not result_dir.exists(): 
        result_dir.mkdir(parents=True) #Make the results directory if it doesn't exist
        
    glob_pattern    = '*' + ext
    file_paths = [path] if path.is_file() else path.glob(glob_pattern)
    
    if result_how == 'concat': 
        profile_result_list = []
        rot_profile_result_list = []
        section_result_list = []
        sections_result_list = []
    
    for file_path in file_paths:
        array = np.loadtxt(file_path,delimiter=delims[file_path.suffix])
        print(file_path)
        
        #--------------Data Cleaning-------------------------
        if level:
            array = plane_level(array)
            print('Got to level')
        if form:
            array = remove_form(array)
            print('Got to form')
        if smooth:
            array = smooth_image(array,sigma=sigma)
            print('got to smooth')
            
        #-------------Parameter Calculation------------------
        if profile:
            profile_results = compute_parameters(array, params1D, profile_mod, to_df = True)
            
            if result_how == 'concat':
                result_list.append(profile_results)
            if gen_rot:
                profiles = gen_rot_prof(array)
                rot_profile_results = compute_parameters(profiles, params1D, profile_mod, to_df = True)
                
                if result_how == 'concat':
                    rot_profile_result_list.append(rot_profile_results)
        
        if section:
            section_results = compute_parameters(array, params2D, section_mod)
            
            if result_how == 'concat':
                section_result_list.append(section_results)
            
            if gen_section:
                sections         = gen_sections(array, how=sec_how, number = sec_num)
                sections_results = compute_parameters(sections, params2D, section_mod, sections = True, to_df = True)

                result_file_name = file_path.stem + '_section' + file_path.suffix #TODO add in .csv and other filer support
                result_path = result_dir / result_file_name 
                print(result_path)
            #np.savetxt(result_path, section_results, delim =delim) #Save the section results

In [ ]:
path = Path()/'nonexistent_file.txt'
pattern = '*.txt'
path.exists()

False

In [ ]:
x = 2
if x == 2: print('its true')

its true


In [ ]:
path = Path.cwd()
path

Path('E:/Archaeology/rough/rough')

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()